In [1]:
import json

import pandas as pd
from rightmove_webscraper import RightmoveData

In [2]:
url = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
rm = RightmoveData(url)

In [3]:
rm.get_results.head()

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,175000.0,4 bedroom town house for sale,"Kiln View, Hanley, Stoke-On-Trent",http://www.rightmove.co.uk/properties/11564120...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,4.0,2022-08-25 12:35:48.809519
1,99950.0,2 bedroom semi-detached house for sale,"Beville Street, Fenton, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12639782...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519
2,95000.0,2 bedroom terraced house for sale,"Whitfield Road, Ball Green",http://www.rightmove.co.uk/properties/12639607...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519
3,205000.0,3 bedroom semi-detached house for sale,"Carsington Drive, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12600143...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,3.0,2022-08-25 12:35:48.809519
4,90000.0,2 bedroom end of terrace house for sale,"Hertford Street, Heron Cross, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12639323...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519


In [4]:
rm.get_results

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,175000.0,4 bedroom town house for sale,"Kiln View, Hanley, Stoke-On-Trent",http://www.rightmove.co.uk/properties/11564120...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,4.0,2022-08-25 12:35:48.809519
1,99950.0,2 bedroom semi-detached house for sale,"Beville Street, Fenton, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12639782...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519
2,95000.0,2 bedroom terraced house for sale,"Whitfield Road, Ball Green",http://www.rightmove.co.uk/properties/12639607...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519
3,205000.0,3 bedroom semi-detached house for sale,"Carsington Drive, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12600143...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,3.0,2022-08-25 12:35:48.809519
4,90000.0,2 bedroom end of terrace house for sale,"Hertford Street, Heron Cross, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12639323...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519
...,...,...,...,...,...,...,...,...,...
545,795000.0,6 bedroom detached house for sale,"Ball Lane, Brown Edge, Staffordshire, ST6",http://www.rightmove.co.uk/properties/11429797...,http://www.rightmove.co.uk/estate-agents/agent...,ST6,NaN,6.0,2022-08-25 12:35:48.809519
546,65000.0,2 bedroom terraced house for sale,"Oldfield Street, Fenton",http://www.rightmove.co.uk/properties/11667558...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-25 12:35:48.809519
547,250000.0,4 bedroom terraced house for sale,"Victoria Road Methodist Church, ST4 2HG",http://www.rightmove.co.uk/properties/11076557...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,ST4 2HG,4.0,2022-08-25 12:35:48.809519
548,115000.0,3 bedroom semi-detached house for sale,"Brookhouse Road, Meir, Stoke-On-Trent, ST3",http://www.rightmove.co.uk/properties/12143530...,http://www.rightmove.co.uk/estate-agents/agent...,ST3,NaN,3.0,2022-08-25 12:35:48.809519


In [357]:
path = 'pop_above_10000'
cities = []

In [358]:
with open(path) as file:
    for line in file:
        cities.append(line.replace("\n", ""))

In [359]:
cities = sorted(cities)
cities

['Aberdare',
 'Aberdeen',
 'Abergavenny',
 'Abertillery',
 'Aberystwyth',
 'Abingdon',
 'Accrington',
 'Adwick le Street',
 'Airdrie',
 'Aldershot',
 'Aldridge',
 'Alloa',
 'Almondbury',
 'Altrincham',
 'Amesbury',
 'Antrim',
 'Appleton',
 'Arbroath',
 'Ardrossan',
 'Armadale',
 'Ashford',
 'Ashton',
 'Aylesbury',
 'Ayr',
 'Banbridge',
 'Bangor',
 'Bangor',
 'Barnsley',
 'Barrhead',
 'Barri',
 'Basford',
 'Basildon',
 'Basingstoke',
 'Bath',
 'Bathgate',
 'Batley',
 'Beaconsfield',
 'Bearsden',
 'Bebington',
 'Bedford',
 'Belfast',
 'Bellshill',
 'Berkhamsted',
 'Beverley',
 'Bickenhill',
 'Billericay',
 'Bingley',
 'Birkenhead',
 'Birmingham',
 'Birstall',
 'Bishop Auckland',
 'Bishopbriggs',
 'Bishops Stortford',
 'Blackburn',
 'Blackpool',
 'Bletchley',
 'Blundellsands',
 'Blunsdon St Andrew',
 'Bodmin',
 'Bognor Regis',
 'Bolton',
 'Bonnyrigg',
 'Bootle',
 'Bournemouth',
 'Bracknell',
 'Bradford',
 'Brentwood',
 'Bridgnorth',
 'Bridlington',
 'Brighouse',
 'Brighton',
 'Brislington

In [407]:
city_encodings = {}

In [408]:
import requests, json
import re
path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'

In [248]:
# test = 'https://www.rightmove.co.uk/typeAhead/uknostreet/BA/SI/NG/ST/OK/E'
# res = requests.get(test).json()
# out = json.dumps(res, indent=2)
# name = "Basingstoke"

In [409]:
# Generate path of the initial results page for every city
for city in cities:
    count = 0
    for char in city:
        if count == 2:
            path += '/'
            count = 0
        if char == "-" or char == " ":
            path += '%20'
        else:
            path += char.upper()
        count += 1

    print(path)
    res = requests.get(path).json()
    out = json.dumps(res, indent=2)


    #searchTerm = re.findall(r'normalisedSearchTerm":(\D+?)}', out)

    # need to exclude station in the capture somehow
    searchTerm = re.findall(r'(?<!STATION\^\d)(?<!STATION\^\d\d)(?<!STATION\^\d\d\d)\D,\DnormalisedSearchTerm":(\D+?)}', out)
    searchTerm = [x.strip() for x in searchTerm]
    print(searchTerm)

    cityJoined = ""

    if " " in city:
        cityJoined = city.replace(" ", "")

    #if this not empty we have match
    resultCities =[x.upper() for x in searchTerm if city.upper() in x or cityJoined.upper() in x]
    print(city.upper())


    if resultCities:
        postfix = resultCities[0]
        reg = 'REGION\^(.+?)",\D+?normalisedSearchTerm\D:\D' + postfix
        print(reg)
        locID = re.search(reg, out)
    else:
        locID = re.search('REGION\^(.+?)"', out)

    if locID:
        print(locID.group(0), locID.group(1))
        found = locID.group(1)

    city_encodings[city] = found
    path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'


https://www.rightmove.co.uk/typeAhead/uknostreet/AB/ER/DA/RE
[]
ABERDARE
REGION^3" 3
https://www.rightmove.co.uk/typeAhead/uknostreet/AB/ER/DE/EN
[]
ABERDEEN
REGION^94487" 94487
https://www.rightmove.co.uk/typeAhead/uknostreet/AB/ER/GA/VE/NN/Y
[]
ABERGAVENNY
REGION^7" 7
https://www.rightmove.co.uk/typeAhead/uknostreet/AB/ER/TI/LL/ER/Y
[]
ABERTILLERY
REGION^10" 10
https://www.rightmove.co.uk/typeAhead/uknostreet/AB/ER/YS/TW/YT/H
[]
ABERYSTWYTH
REGION^11" 11
https://www.rightmove.co.uk/typeAhead/uknostreet/AB/IN/GD/ON
[]
ABINGDON
REGION^12" 12
https://www.rightmove.co.uk/typeAhead/uknostreet/AC/CR/IN/GT/ON
[]
ACCRINGTON
REGION^14" 14
https://www.rightmove.co.uk/typeAhead/uknostreet/AD/WI/CK/%20L/E%20/ST/RE/ET
[]
ADWICK LE STREET
REGION^1704" 1704
https://www.rightmove.co.uk/typeAhead/uknostreet/AI/RD/RI/E
[]
AIRDRIE
REGION^18" 18
https://www.rightmove.co.uk/typeAhead/uknostreet/AL/DE/RS/HO/T
[]
ALDERSHOT
REGION^22" 22
https://www.rightmove.co.uk/typeAhead/uknostreet/AL/DR/ID/GE
[]
ALDRID

In [414]:
path = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

In [415]:
city_encodings, len(city_encodings)

({'Aberdare': '3',
  'Aberdeen': '94487',
  'Abergavenny': '7',
  'Abertillery': '10',
  'Aberystwyth': '11',
  'Abingdon': '12',
  'Accrington': '14',
  'Adwick le Street': '1704',
  'Airdrie': '18',
  'Aldershot': '22',
  'Aldridge': '1824',
  'Alloa': '27',
  'Almondbury': '1900',
  'Altrincham': '33',
  'Amesbury': '1985',
  'Antrim': '2060',
  'Appleton': '2083',
  'Arbroath': '45',
  'Ardrossan': '47',
  'Armadale': '2185',
  'Ashford': '57',
  'Ashton': '61',
  'Aylesbury': '72',
  'Ayr': '74',
  'Banbridge': '94',
  'Bangor': '98',
  'Barnsley': '108',
  'Barrhead': '2951',
  'Barri': '2953',
  'Basford': '93007',
  'Basildon': '115',
  'Basingstoke': '115',
  'Bath': '116',
  'Bathgate': '117',
  'Batley': '118',
  'Beaconsfield': '120',
  'Bearsden': '3128',
  'Bebington': '3142',
  'Bedford': '129',
  'Belfast': '133',
  'Bellshill': '135',
  'Berkhamsted': '141',
  'Beverley': '146',
  'Bickenhill': '3439',
  'Billericay': '155',
  'Bingley': '159',
  'Birkenhead': '161',
 

In [416]:
freq = {}
for key, value in city_encodings.items():
    freq.setdefault(value, set()).add(key)

In [417]:
[(value, key) for key, value in freq.items() if len(value) > 1]

[({'Basildon', 'Basingstoke'}, '115'),
 ({'Bury', 'Bury Saint Edmunds'}, '258'),
 ({'Kingston upon Hull', 'Kingston upon Thames'}, '746'),
 ({'Milton', 'Milton Keynes'}, '940'),
 ({'Oswaldtwistle', 'Oswestry'}, '1033'),
 ({'Plymouth', 'Plymstock'}, '1073'),
 ({'Royal Leamington Spa', 'Royal Tunbridge Wells'}, '94454'),
 ({'Shenley Brook End', 'Shenley Church End'}, '21940'),
 ({'Stratford', 'Stratford-upon-Avon'}, '1282'),
 ({'Walsall', 'Walsall Wood'}, '1392'),
 ({'Woking', 'Wokingham'}, '1474')]

In [ ]:
data = []
counts = {}

for city in cities:
    encoding = city_encodings[city]
    old = path[path.index("REGION%"): path.index("&")]
    new = path.replace(old, "REGION^" + encoding)
    rm = RightmoveData(new)
    data.append(rm.get_results)
    counts[city] = len(rm.get_results)
    print(city, " ", counts[city])


Aberdare   105
Aberdeen   88
Abergavenny   40
Abertillery   46
Aberystwyth   39
Abingdon   82
Accrington   146
Adwick le Street   20
Airdrie   35
Aldershot   74
Aldridge   46
Alloa   15
Almondbury   13
Altrincham   240
Amesbury   38
Antrim   5
Appleton   79
Arbroath   17
Ardrossan   5
Armadale   11
Ashford   293
Ashton   120
Aylesbury   270
Ayr   69
Banbridge   176
Bangor   82
Bangor   82
Barnsley   572
Barrhead   15
Barri   9
Basford   44
Basildon   149
Basingstoke   149
Bath   239
Bathgate   21
Batley   87
Beaconsfield   99
Bearsden   20
Bebington   57
Bedford   228
Belfast   10
Bellshill   10
Berkhamsted   75
Beverley   181
Bickenhill   0
Billericay   141
Bingley   59
Birkenhead   139


In [80]:
import pandas as pd
result = pd.concat(data)
result

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,140000.0,2 bedroom semi-detached house for sale,"John Street Way, Wombwell, S73 8FT - Viewing E...",http://www.rightmove.co.uk/properties/85815972...,http://www.rightmove.co.uk/estate-agents/agent...,S73,S73 8FT,2.0,2022-08-18 17:19:27.506480
1,290000.0,3 bedroom detached house for sale,"Montrose Avenue,Darton,Barnsley,S75 5LS",http://www.rightmove.co.uk/properties/12537744...,http://www.rightmove.co.uk/estate-agents/agent...,S75,S75 5LS,3.0,2022-08-18 17:19:27.506480
2,400000.0,3 bedroom detached house for sale,"Rotherham Road, Monk Bretton, S71",http://www.rightmove.co.uk/properties/12617442...,http://www.rightmove.co.uk/estate-agents/agent...,S71,NaN,3.0,2022-08-18 17:19:27.506480
3,110000.0,3 bedroom terraced house for sale,"Pontefract Road, Oakwell",http://www.rightmove.co.uk/properties/10952320...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,3.0,2022-08-18 17:19:27.506480
4,325000.0,5 bedroom semi-detached house for sale,"Park Grove, Barnsley, S70",http://www.rightmove.co.uk/properties/12616609...,http://www.rightmove.co.uk/estate-agents/agent...,S70,NaN,5.0,2022-08-18 17:19:27.506480
...,...,...,...,...,...,...,...,...,...
277,365000.0,5 bedroom semi-detached house for sale,"Cherry Wood Crescent, Fulford, York",http://www.rightmove.co.uk/properties/85711197...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,5.0,2022-08-18 17:32:55.035881
278,1500000.0,5 bedroom town house for sale,"St. Martins Lane, York, YO1 6LN",http://www.rightmove.co.uk/properties/11392433...,http://www.rightmove.co.uk/estate-agents/agent...,YO1,YO1 6LN,5.0,2022-08-18 17:32:55.035881
279,1595000.0,10 bedroom semi-detached house for sale,"St. George's Place, York, YO24",http://www.rightmove.co.uk/properties/81442933...,http://www.rightmove.co.uk/estate-agents/agent...,YO24,NaN,10.0,2022-08-18 17:32:55.035881
280,1500000.0,5 bedroom house for sale,"St. Martins Lane, York, North Yorkshire, YO1",http://www.rightmove.co.uk/properties/10194097...,http://www.rightmove.co.uk/estate-agents/agent...,YO1,NaN,5.0,2022-08-18 17:32:55.035881


In [ ]:
# Decide on city count
# Limit cities with listings minimum threshold (200?)
# Use google api to get accurate lat long from the right move address (+ use postcode if they have one) https://www.codingforentrepreneurs.com/blog/python-tutorial-google-geocoding-api/
# or use worldcities.csv
#